# Scheduler = service to start pipeline at specified times

# Orchestrator = System to ensure tasks in a data pipeline are run in the correct order 

# Airflow is both a scheduler and an orchestrator

# Define data pipeline as a DAG

In [1]:
%%capture
! docker compose down

In [2]:
! docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


In [3]:
# do this in a terminal in this directory
# sudo mkdir -p logs plugins temp dags tests data visualization && sudo chmod -R u=rwx,g=rwx,o=rwx logs plugins temp dags tests data visualization

In [10]:
%%capture
! docker compose up --build -d

In [11]:
! sleep 30

In [12]:
! rm -rf ./dags/tpch_warehouse/models/*/.ipynb_checkpoints # always run before dbt run, caused by notebooks, no need to do this if performed via terminal

zsh:1: no matches found: ./dags/tpch_warehouse/models/*/.ipynb_checkpoints


In [13]:
! docker ps

CONTAINER ID   IMAGE                                           COMMAND                  CREATED          STATUS                    PORTS                                       NAMES
96e73ce0515e   metabase/metabase                               "/app/run_metabase.sh"   33 seconds ago   Up 31 seconds             0.0.0.0:3000->3000/tcp, :::3000->3000/tcp   dashboard
dd6ea004cf36   6-scheduling--orchestration-airflow-webserver   "/usr/bin/dumb-init …"   52 minutes ago   Up 51 minutes (healthy)   0.0.0.0:8080->8080/tcp, :::8080->8080/tcp   webserver
3b5d361b77a5   6-scheduling--orchestration-airflow-scheduler   "/usr/bin/dumb-init …"   52 minutes ago   Up 52 minutes (healthy)   8080/tcp                                    scheduler
827ce29a3da1   postgres:16                                     "docker-entrypoint.s…"   52 minutes ago   Up 52 minutes (healthy)   0.0.0.0:5432->5432/tcp, :::5432->5432/tcp   postgres


In [ ]:
%%capture
! docker compose down